In [ ]:
from build_nba_bdd import export_all_seasons, export_all_teams_seasons, export_all_salaries, scrape_mvp_data

In [ ]:
export_all_seasons()

In [ ]:
export_all_teams_seasons()

In [ ]:
export_all_salaries()

In [ ]:
scrape_mvp_data()